<a href="https://colab.research.google.com/github/sarathpanat/Sentiment-classification/blob/main/albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#sentiment classification using hugging face transformers

In [ ]:
#1.install packages

In [ ]:
!pip install -q kaggle
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 1.8MB 8.3MB/s 
     |████████████████████████████████| 2.9MB 50.8MB/s 
     |████████████████████████████████| 890kB 56.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d3f057ac19d7b292c1694a579581d7c6591838ad410d80f6f1d0212c87d9d1f8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 8.2MB/s 


In [ ]:
#2.connect kaggle with colab and import data

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sarathpanat","key":"b934fc5effb401610cfe8a41e461b02a"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s 'Sentiment140 dataset with 1.6 million tweets'

ref                                                    title                                          size  lastUpdated          downloadCount  
-----------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  
kazanova/sentiment140                                  Sentiment140 dataset with 1.6 million tweets   81MB  2017-09-13 22:43:19          47194  
grwche/sentiment                                       Sentiment                                       2GB  2020-04-23 23:29:23            100  
raohammas/sentimental-analysis-trained-model-on-mlnet  Sentimental Analysis trained Model on ML.Net  120MB  2020-10-28 18:25:53              9  
malikhasan/16-million-sentiment-tweets                 1.6 million sentiment tweets                   81MB  2020-04-14 22:54:15             14  


In [ ]:
!kaggle datasets download -d 'kazanova/sentiment140'
!unzip 'sentiment140.zip'

 85% 69.0M/80.9M [00:00<00:00, 45.2MB/s]
100% 80.9M/80.9M [00:00<00:00, 98.0MB/s]
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
import pandas as pd
import numpy as np

names= ["sentiment","id","date","flag","user","text"]
encoding = "ISO-8859-1"
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding=encoding ,names=names)

In [ ]:
df = df.drop(["id","date","flag","user"],axis=1)


In [ ]:
df

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['sentiment'] = df.sentiment.replace(4,1)
df

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
#3.import model -albert

In [ ]:
from transformers import AlbertTokenizer, TFAlbertModel
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
albert = TFAlbertModel.from_pretrained("albert-base-v2")

Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [ ]:
print(tokenizer), albert.summary()

PreTrainedTokenizer(name_or_path='albert-base-v2', vocab_size=30000, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
Model: "tf_albert_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
Total params: 11,683,584
Trainable params: 11,683,584
Non-trainable params: 0
_________________________________________________________________


(None, None)

In [ ]:
tokenizer("i am member of ai club")

{'input_ids': [2, 31, 589, 322, 16, 21, 49, 288, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#4.data preparation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], train_size=0.02, test_size=0.01, shuffle=True, stratify=df['sentiment'])

In [ ]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, train_size=0.75, shuffle=True, stratify=y_test)

In [ ]:
y_train.value_counts()

1    16000
0    16000
Name: sentiment, dtype: int64

In [ ]:
train_ids= []
train_att = []

for i in X_train:
  encoded_data = tokenizer.encode_plus(
            i,
            add_special_tokens=True,
            max_length=128, # truncates if len(s) > max_length
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
  )
  train_ids.append(encoded_data['input_ids'])
  train_att.append(encoded_data['attention_mask'])

train_ids = np.array(train_ids)
train_att = np.array(train_att)

In [ ]:
train_ids= []
train_att = []

for i in X_train:
  encoded_data = tokenizer.encode_plus(
            i,
            add_special_tokens=True,
            max_length=128, # truncates if len(s) > max_length
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
  )
  train_ids.append(encoded_data['input_ids'])
  train_att.append(encoded_data['attention_mask'])

train_ids = np.array(train_ids)
train_att = np.array(train_att)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
test_ids= []
test_att = []

for i in X_test:
  encoded_data = tokenizer.encode_plus(
            i,
            add_special_tokens=True,
            max_length=128, # truncates if len(s) > max_length
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
  )
  test_ids.append(encoded_data['input_ids'])
  test_att.append(encoded_data['attention_mask'])

test_ids = np.array(test_ids)
test_att = np.array(test_att)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
valid_ids= []
valid_att = []

for i in X_valid:
  encoded_data = tokenizer.encode_plus(
            i,
            add_special_tokens=True,
            max_length=128, # truncates if len(s) > max_length
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
  )
  valid_ids.append(encoded_data['input_ids'])
  valid_att.append(encoded_data['attention_mask'])

valid_ids = np.array(valid_ids)
valid_att = np.array(valid_att)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#5.fine turning model 

In [ ]:
import tensorflow as tf


In [ ]:
def build_model(mod):

  ids=tf.keras.layers.Input((128,),dtype=tf.int32)
  att=tf.keras.layers.Input((128,),dtype=tf.int32)

  x=mod(ids, attention_mask=att)

  layer1 = tf.keras.layers.Dense(128, activation='relu')(x[1])
  drop1 = tf.keras.layers.Dropout(0.4)(layer1)
  layer2 = tf.keras.layers.Dense(64, activation='relu')(drop1)
  drop2 = tf.keras.layers.Dropout(0.4)(layer2)
  layer3 = tf.keras.layers.Dense(64, activation='relu')(drop2)
  drop3 = tf.keras.layers.Dropout(0.4)(layer3)
  output = tf.keras.layers.Dense(1, activation='sigmoid')(drop3)

  model = tf.keras.Model(inputs = [ids,att], outputs=[output])
  #model.compile(optimizer=tf.keras.optimizers.Adam(3e-5),loss='binary_crossentropy',metrics=tf.keras.metrics.BinaryAccuracy())
  return model


In [ ]:
model2 = build_model(albert)
model2.compile(optimizer=tf.keras.optimizers.Adam(3e-5),loss='binary_crossentropy',metrics=tf.keras.metrics.BinaryAccuracy())

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_albert_model_1 (TFAlbertMode TFBaseModelOutputWit 11683584    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 128)          98432       tf_albert_model_1[1][1]      

In [ ]:
#6.training and prediction

In [ ]:
hist = model2.fit(x=[train_ids[:10],train_att[:10]],y=[y_train[:10]], epochs=10, batch_size=32, verbose=1, validation_data=([valid_ids[:10],valid_att[:10]],y_valid[:10]))

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/10


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1/1 [==============================] - ETA: 0s - loss: 0.9018 - binary_accuracy: 0.3000

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1/1 [==============================] - 15s 15s/step - loss: 0.9018 - binary_accuracy: 0.3000 - val_loss: 0.6852 - val_binary_accuracy: 0.6000
Epoch 2/10
1/1 [==============================] - 0s 439ms/step - loss: 0.9000 - binary_accuracy: 0.5000 - val_loss: 0.6954 - val_binary_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 393ms/step - loss: 0.7282 - binary_accuracy: 0.5000 - val_loss: 0.6940 - val_binary_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 403ms/step - loss: 0.5327 - binary_accuracy: 0.7000 - val_loss: 0.6961 - val_binary_accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 400ms/step - loss: 0.6647 - binary_accuracy: 0.7000 - val_loss: 0.6819 - val_binary_accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 0s 395ms/step - loss: 1.0512 - binary_accuracy: 0.2000 - val_loss: 0.6727 - val_binary_accuracy: 0.4000
Epoch 7/10
1/1 [==============================] - 0s 401ms/step - loss: 0.6598 - binary_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model2.load_weights('/content/drive/MyDrive/Colab Notebooks/Albert_.h5')

In [ ]:
predict = model2.predict([test_ids[:10], test_att[:10]],batch_size=32)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
predict= predict.round()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test[:10], predict)

In [ ]:
accuracy

0.8